In [74]:
!pip install pyproj

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pyproj
from datetime import datetime, timedelta
from sgp4.api import Satrec, WGS72
from sgp4.api import jday

In [3]:
def load_tle_data(tle_file):
    satellites = []
    with open(tle_file, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            name = lines[i].strip()
            line1 = lines[i+1].strip()
            line2 = lines[i+2].strip()
            satellites.append([name, line1, line2])
    return satellites
print(load_tle_data('30sats.txt')[1])

['AEROCUBE 12A', '1 43556U 18046C   23055.30616910  .00104379  00000+0  13224-2 0  9990', '2 43556  51.6307  23.8082 0003431 184.7728 175.3232 15.58641168258270']


In [4]:
def get_satellite_position(satellite, times):
    positions = []
    sat = Satrec.twoline2rv(satellite[1], satellite[2], WGS72)
    for t in times:
        jd, fr = jday(2024, 6, 10, 0, 0, 0 + t / 1440.0 * 24)
        e, r, v = sat.sgp4(jd, fr)
        if e == 0:
            positions.append((t, r, v))
            # print(positions)
    return positions
sat1 = get_satellite_position(load_tle_data('30sats.txt')[1], [0]) 
print(sat1)

[]


In [5]:
def ecef2lla(pos_x, pos_y, pos_z):
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")
    lona, lata, alta = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
    return lona, lata, alta

In [6]:
def process_satellite(sat, times):
    positions = get_satellite_position(sat, times)
    pos_array = np.array(positions, dtype=object)
    # Debugging print statement to check pos_array shape
    # print(f"pos_array shape: {pos_array.shape}")
    lx = []
    ly = []
    lz = []
    try :
        for l in pos_array[:, 1]:
            lx.append(l[0])
            ly.append(l[1])
            lz.append(l[2])
    except IndexError:
        return []
    lon, lat, alt = ecef2lla(lx, ly, lz)
    return [lon, lat, alt]

In [7]:
def filter_positions(positions, region_coords):
    filtered_positions = []
    for pos in positions:
        if pos[7] >= region_coords[0][0] and pos[7] <= region_coords[1][0] and pos[8] >= region_coords[2][1] and pos[8] <= region_coords[3][1]:
            filtered_positions.append(pos)
    return filtered_positions

In [ ]:
import time
start_time = time.time()

if __name__ == "__main__":
    tle_file = '30000sats.txt'
    times = np.arange(0, 1440)  # One minute intervals for one day
    region_coords = [(16.66673, 103.58196), (69.74973, -120.64459), (-21.09096, -119.71009), (-31.32309, -147.79778)]
    satellites = load_tle_data(tle_file)
    results = [process_satellite(sat, times) for sat in satellites]
    filtered_results = [filter_positions(res, region_coords) for res in results]

    print(f"Number of satellites in region: {len(filtered_results)}")
print("--- %s seconds ---" % (time.time() - start_time))


C:\Users\Abhinav\AppData\Local\Temp\ipykernel_23352\44107315.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lona, lata, alta = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
C:\Users\Abhinav\AppData\Local\Temp\ipykernel_23352\44107315.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lona, lata, alta = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
C:\Users\Abhinav\AppData\Local\Temp\ipykernel_23352\44107315.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lona, lata, alta = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
C:\Users\Abhinav\AppData\Local\Temp\ipykernel_23352\44107315.py:4: FutureWarning: This function is deprecated. See: https://pyproj